<a href="https://colab.research.google.com/github/ashishpatel26/NER-with-SpanMarker/blob/main/1.%20SpanMarker%20for%20Named%20Entity%20Recognition%20Beginner%20Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting Started with SpanMarker
[SpanMarker](https://github.com/tomaarsen/SpanMarkerNER) is an accessible yet powerful Python module for training Named Entity Recognition models.

In this notebook, we'll have a look at how to train an NER model using SpanMarker.

### Setup
First of all, the `span_marker` Python module needs to be installed. If we want to use [Weights and Biases](https://wandb.ai/) for logging, we can install `span_marker` using the `[wandb]` extra.

In [1]:
%pip install span_marker
# %pip install span_marker[wandb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:0

### Loading the dataset
For this example, we'll load the challenging [FewNERD supervised dataset](https://huggingface.co/datasets/DFKI-SLT/few-nerd) from the Hugging Face hub using 🤗 Datasets.

In [2]:
from datasets import load_dataset

dataset = load_dataset("DFKI-SLT/few-nerd", "supervised")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/3359329 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

  0%|          | 0/482037 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

  0%|          | 0/958765 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 131767
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 18824
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 37648
    })
})

Let's inspect some samples to get a feel for the data.

In [3]:
for sample in dataset["train"].select(range(3)):
    print(sample)

{'id': '0', 'tokens': ['Paul', 'International', 'airport', '.'], 'ner_tags': [0, 0, 0, 0], 'fine_ner_tags': [0, 0, 0, 0]}
{'id': '1', 'tokens': ['It', 'starred', 'Hicks', "'s", 'wife', ',', 'Ellaline', 'Terriss', 'and', 'Edmund', 'Payne', '.'], 'ner_tags': [0, 0, 7, 0, 0, 0, 7, 7, 0, 7, 7, 0], 'fine_ner_tags': [0, 0, 51, 0, 0, 0, 50, 50, 0, 50, 50, 0]}
{'id': '2', 'tokens': ['``', 'Time', '``', 'magazine', 'said', 'the', 'film', 'was', '``', 'a', 'multimillion', 'dollar', 'improvisation', 'that', 'does', 'everything', 'but', 'what', 'the', 'title', 'promises', "''", 'and', 'suggested', 'that', '``', 'writer', 'George', 'Axelrod', '(', '``', 'The', 'Seven', 'Year', 'Itch', '``', ')', 'and', 'director', 'Richard', 'Quine', 'should', 'have', 'taken', 'a', 'hint', 'from', 'Holden', '[', "'s", 'character', 'Richard', 'Benson', ']', ',', 'who', 'writes', 'his', 'movie', ',', 'takes', 'a', 'long', 'sober', 'look', 'at', 'what', 'he', 'has', 'wrought', ',', 'and', 'burns', 'it', '.', "''"], 'n

As you can see, this dataset contains `tokens`, `ner_tags` and a `fine_ner_tags` columns. Let's have a look at which labels these last two represent using the [Dataset features](https://huggingface.co/docs/datasets/about_dataset_features).

In [16]:
labels = dataset["train"].features["ner_tags"].feature.names
print(labels)

['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product']


In [14]:
fine_labels = dataset["train"].features["fine_ner_tags"].feature.names
print(fine_labels)

['O', 'art-broadcastprogram', 'art-film', 'art-music', 'art-other', 'art-painting', 'art-writtenart', 'building-airport', 'building-hospital', 'building-hotel', 'building-library', 'building-other', 'building-restaurant', 'building-sportsfacility', 'building-theater', 'event-attack/battle/war/militaryconflict', 'event-disaster', 'event-election', 'event-other', 'event-protest', 'event-sportsevent', 'location-GPE', 'location-bodiesofwater', 'location-island', 'location-mountain', 'location-other', 'location-park', 'location-road/railway/highway/transit', 'organization-company', 'organization-education', 'organization-government/governmentagency', 'organization-media/newspaper', 'organization-other', 'organization-politicalparty', 'organization-religion', 'organization-showorganization', 'organization-sportsleague', 'organization-sportsteam', 'other-astronomything', 'other-award', 'other-biologything', 'other-chemicalthing', 'other-currency', 'other-disease', 'other-educationaldegree', '

For the purposes of this tutorial, let's stick with the `ner_tags` coarse-grained labels, but I challenge you to modify this Notebook to train for the fine labels. For the SpanMarker model, any dataset can be used as long as it has a `tokens` and a `ner_tags` column. The `ner_tags` can be annotated using the IOB, IOB2, BIOES or BILOU labeling scheme, but also regular unschemed labels like in this FewNERD example can be used.

### Initializing a `SpanMarkerModel`
A SpanMarker model is initialized via [SpanMarkerModel.from_pretrained](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.modeling.html#span_marker.modeling.SpanMarkerModel.from_pretrained). This method will be familiar to those who know 🤗 Transformers. It accepts either a path to a local model or the name of a model on the [Hugging Face Hub](https://huggingface.co/models).

Importantly, the model can *either* be an encoder or an already trained and saved SpanMarker model. As we haven't trained anything yet, we will use an encoder. To learn how to load and use a saved SpanMarker model, please have a look at the [Loading & Inferencing](model_loading.ipynb) notebook.

Reasonable options for encoders include BERT, RoBERTa, etc., which means that the following are all good options: `"bert-base-cased"`, `"bert-large-cased"`, `"roberta-base"`, `"roberta-large"`. Not all encoders work though, they **must** allow for `position_ids` as an input argument, which disqualifies DistilBERT, T5, DistilRoBERTa, ALBERT & BART. Furthermore, using uncased models is generally not recommended, as the capitalisation can be very useful to find named entities.

We'll use `"bert-base-cased"` for this notebook. If you're running this on Google Colab, be sure to set hardware accelerator to "GPU" in `Runtime` > `Change runtime type`.

In [17]:
from span_marker import SpanMarkerModel

model_name = "bert-base-cased"
model = SpanMarkerModel.from_pretrained(model_name, labels=labels, model_max_length=256)

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 28998. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


For us, these warnings are expected, as we are initializing `BertModel` for a new task.

Note that we provided `SpanMarkerModel.from_pretrained` with a list of our labels. This is required when training a new model using an encoder. Furthermore, we can specify some useful configuration parameters from `SpanMarkerConfig`, such as:

* `model_max_length`: The maximum number of tokens that the model will process. If you only use short sentences for your model, reducing this number may help training and inference speeds with no loss in performance. Defaults to the encoder maximum, or 512 if the encoder doesn't have a maximum.
* `entity_max_length`: The total number of words that one entity can be. Defaults to 16.

### Training
At this point, our model is already ready for training! We can import [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) directly from 🤗 Transformers as well as the SpanMarker `Trainer`. The `Trainer` is a subclass of the 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) that simplifies some tasks for you, but otherwise it works just like the regular `Trainer`.

This next snippet shows some reasonable defaults. Feel free to adjust the batch size to a lower value if you experience out of memory exceptions.

In [18]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/span-marker-bert-base-fewnerd-coarse-super",
    learning_rate=5e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    push_to_hub=False,
    logging_steps=50,
    warmup_ratio=0.1,
    dataloader_num_workers=2,
)

Now we can create a SpanMarker `Trainer` in the same way that you would initialize a 🤗 Transformers `Trainer`. We'll train on a subsection of the data to save us some time. Amazingly, this `Trainer` will automatically create logs using exactly the logging tools that you have installed. With other words, if you prefer logging with [Tensorboard](https://www.tensorflow.org/tensorboard), all that you have to do is install it.

In [19]:
from span_marker import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"].select(range(8000)),
    eval_dataset=dataset["validation"].select(range(2000)),
)

INFO:span_marker.label_normalizer:No labeling scheme detected: all label IDs belong to individual entity classes.


Let's start training!

In [20]:
trainer.train()

Label normalizing the train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing the train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

These are the frequencies of the missed entities due to maximum entity length out of 20351 total entities:
- 24 missed entities with 9 words (0.117930%)
- 15 missed entities with 10 words (0.073706%)
- 14 missed entities with 11 words (0.068793%)
- 7 missed entities with 12 words (0.034396%)
- 5 missed entities with 13 words (0.024569%)
- 2 missed entities with 15 words (0.009828%)
- 1 missed entities with 17 words (0.004914%)
- 1 missed entities with 19 words (0.004914%)


Spreading data between multiple samples:   0%|          | 0/8000 [00:00<?, ? examples/s]

INFO:span_marker.trainer:Spread 8000 sentences across 9818 samples, a 22.725000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.


...

Tokenizing the evaluation dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

These are the frequencies of the missed entities due to maximum entity length out of 5203 total entities:
- 5 missed entities with 9 words (0.096098%)
- 5 missed entities with 10 words (0.096098%)
- 2 missed entities with 11 words (0.038439%)
- 1 missed entities with 12 words (0.019220%)
- 3 missed entities with 13 words (0.057659%)


Spreading data between multiple samples:   0%|          | 0/2000 [00:00<?, ? examples/s]

INFO:span_marker.trainer:Spread 2000 sentences across 2449 samples, a 22.450000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.
INFO:span_marker.trainer:Spread 2000 sentences across 2449 samples, a 22.450000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.
INFO:span_marker.trainer:Spread 2000 sentences across 2449 samples, a 22.450000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.
INFO:span_marker.trainer:Spread 2000 sentences across 2449 samples, a 22.450000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.


TrainOutput(global_step=1227, training_loss=0.07208207932155893, metrics={'train_runtime': 1611.6375, 'train_samples_per_second': 6.092, 'train_steps_per_second': 0.761, 'total_flos': 2583068872089600.0, 'train_loss': 0.07208207932155893, 'epoch': 1.0})

And now the final step is to compute the model's performance.

In [21]:
metrics = trainer.evaluate()
metrics

INFO:span_marker.trainer:Spread 2000 sentences across 2449 samples, a 22.450000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.


{'eval_loss': 0.019086642190814018,
 'eval_overall_precision': 0.7816790898391526,
 'eval_overall_recall': 0.7682668208983998,
 'eval_overall_f1': 0.774914924647545,
 'eval_overall_accuracy': 0.9393945575455918,
 'eval_runtime': 94.4964,
 'eval_samples_per_second': 25.916,
 'eval_steps_per_second': 6.487,
 'epoch': 1.0}

Additionally, we should evaluate using the test set.

In [23]:
trainer.evaluate(dataset["test"].select(range(2000)), metric_key_prefix="test")

INFO:span_marker.trainer:Spread 2000 sentences across 2440 samples, a 22.000000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.


{'test_loss': 0.01875729113817215,
 'test_overall_precision': 0.7872506843957763,
 'test_overall_recall': 0.7687607408821845,
 'test_overall_f1': 0.7778958554729013,
 'test_overall_accuracy': 0.9398919359771638,
 'test_runtime': 94.0036,
 'test_samples_per_second': 25.956,
 'test_steps_per_second': 6.489,
 'epoch': 1.0}

Let's try the model out with some predictions. For this we can use the `model.predict` method, which accepts either:

* A sentence as a string.
* A tokenized sentence as a list of strings.
* A list of sentences as a list of strings.
* A list of tokenized sentences as a list of lists of strings.

The method returns a list of dictionaries for each sentence, with the following keys:

* `"label"`: The string label for the found entity.
* `"score"`: The probability score indicating the model its confidence.
* `"span"`: The entity span as a string.
* `"word_start_index"` and `"word_end_index"`: Integers useful for indexing the entity from a tokenized sentence.
* `"char_start_index"` and `"char_end_index"`: Integers useful for indexing the entity from a string sentence.

In [24]:
sentences = [
    "The Ninth suffered a serious defeat at the Battle of Camulodunum under Quintus Petillius Cerialis in the rebellion of Boudica (61), when most of the foot-soldiers were killed in a disastrous attempt to relieve the besieged city of Camulodunum (Colchester).",
    "He was born in Wellingborough, Northamptonshire, where he attended Victoria Junior School, Westfield Boys School and Sir Christopher Hatton School.",
    "Nintendo continued to sell the revised Wii model and the Wii Mini alongside the Wii U during the Wii U's first release year.",
    "Dorsa has a Bachelor of Music in Composition from California State University, Northridge in 2001, Master of Music in Harpsichord Performance at Cal State Northridge in 2004, and a Doctor of Musical Arts at the University of Michigan, Ann Arbor in 2008."
]

entities_per_sentence = model.predict(sentences)

for entities in entities_per_sentence:
    for entity in entities:
        print(entity["span"], "=>", entity["label"])
    print()

Battle of Camulodunum => event
Quintus Petillius Cerialis => person
Boudica => location
Camulodunum => location
Colchester => location

Wellingborough => location
Northamptonshire => location
Victoria Junior School => organization
Westfield Boys School => organization
Christopher Hatton School => organization

Nintendo => organization
Wii => product
Wii Mini => product
Wii U => product
Wii U => product

Dorsa => person
Bachelor of Music in Composition => other
California State University => organization
Northridge => organization
Master of Music in Harpsichord Performance => other
Cal State Northridge => organization
Doctor of Musical Arts => other
University of Michigan => organization
Ann Arbor => location



Very impressive performance for less than 8 minutes trained! 🎉

Once trained, we can save our new model locally.

In [25]:
trainer.save_model("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

Or we can push it to the 🤗 Hub like so. I've commented it away for now to prevent people from accidentally pushing models.

In [28]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("ashishpatel26/bloomz-560m-tagger",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

In [33]:
trainer.push_to_hub("ashishpatel26/span-marker-bert-base-fewnerd-coarse-super")

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

'https://huggingface.co/ashishpatel26/span-marker-bert-base-fewnerd-coarse-super/tree/main/'

If we want to use it again, we can just load it using the checkpoint or using the model name on the Hub. This is how it would be done using a local checkpoint.

In [34]:
model = SpanMarkerModel.from_pretrained("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 28998. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


That was all! As simple as that. If we put it all together into a single script, it looks something like this:
```python
from datasets import load_dataset
from span_marker import SpanMarkerModel, Trainer
from transformers import TrainingArguments

def main():
    dataset = load_dataset("DFKI-SLT/few-nerd", "supervised")
    labels = dataset["train"].features["ner_tags"].feature.names

    model_name = "bert-base-cased"
    model = SpanMarkerModel.from_pretrained(model_name, labels=labels)

    args = TrainingArguments(
        output_dir="models/span-marker-bert-base-fewnerd-coarse-super",
        learning_rate=5e-5,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        push_to_hub=False,
        logging_steps=50,
        warmup_ratio=0.1,
        dataloader_num_workers=2,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"].select(range(8000)),
        eval_dataset=dataset["validation"].select(range(2000)),
    )

    trainer.train()
    trainer.save_model("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

    metrics = trainer.evaluate()
    print(metrics)

if __name__ == "__main__":
    main()
```

With `wandb` initialized, you can enjoy their very useful training graphs straight in your browser. It ends up looking something like this.
![image](https://user-images.githubusercontent.com/37621491/235196250-15d595f4-6d72-4625-bde9-f3783484997b.png)
![image](https://user-images.githubusercontent.com/37621491/235196335-6f36a7fb-5274-4ce5-a1f3-1d2ad35b26a4.png)

Furthermore, you can use the `wandb` hyperparameter search functionality using the tutorial from the Hugging Face documentation [here](https://huggingface.co/docs/transformers/hpo_train). This transfers very well to the SpanMarker `Trainer`.